In [5]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd         
import tqdm
import time 

In [14]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import tqdm
import time 

url = "https://dantri.com.vn/kinh-doanh/chung-khoan/trang-"
page = 2
base_url = url
articles = []  

while True:
    try:
        url = base_url + str(page) + ".htm"

        page += 1
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        target = soup.find('div', class_='article list')

        # break code if there is no more article
        if target == None:
            break

        for i in tqdm.tqdm(target.find_all('h3')):
            article_data = {
                'link': "https://dantri.com.vn" + i.find('a')['href'],
                'title': i.text.strip(),
                'time' : [],
                'content': []
            }
            response = requests.get(article_data['link'])
            article_soup = BeautifulSoup(response.text, 'html.parser')
            article_data['time'] = article_soup.find('time', class_='author-time').text.strip().split(" ")[2]
            article_content = article_soup.find('div', class_='singular-content')   
            if article_content:
                for p in article_content.find_all('p'):
                    article_data['content'].append(p.text.strip())
                
                article_data['content'] = " ".join(article_data['content'])    

            articles.append(article_data)


    except Exception as e:
        print(e)
        continue

100%|██████████| 20/20 [00:10<00:00,  1.84it/s]


In [16]:
data = pd.DataFrame(articles)
data.head()

,link,title,time,content
0,https://dantri.com.vn/kinh-doanh/nvidia-cong-b...,"Nvidia công bố doanh thu đột biến nhờ AI, cổ p...",23/05/2024,"Kết phiên giao dịch ngày 22/5 (theo giờ Mỹ), c..."
1,https://dantri.com.vn/kinh-doanh/xang-len-gia-...,"Xăng lên giá, cổ phiếu xăng dầu bùng nổ",23/05/2024,Rung lắc mạnh trong phiên sáng và tiếp tục điề...
2,https://dantri.com.vn/kinh-doanh/dai-thieu-gia...,Đại thiếu gia nhà bầu Hiển tăng sở hữu tại SHB,23/05/2024,Ông Đỗ Quang Vinh - Phó chủ tịch Hội đồng quản...
3,https://dantri.com.vn/kinh-doanh/co-phieu-tra-...,"""Cổ phiếu trà đá"" tăng ồ ạt, một mã ngân hàng ...",21/05/2024,"Trong phiên 21/5, chỉ số chính VN-Index điều c..."
4,https://dantri.com.vn/kinh-doanh/co-phieu-nha-...,"Cổ phiếu nhà Cường ""Đô La"" trở lại; tiền ồ ạt ...",20/05/2024,Mặc dù rung lắc và giằng co nhưng trong suốt p...


In [17]:
data.to_csv('data_dantri_chungkhoan.csv', index=None, encoding='utf-8-sig')

In [18]:
data.shape

(580, 4)

In [51]:
import os, sys, gc, warnings
import logging, math, re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell

# These settings help in proper formatting and display of the output of code we run
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


InteractiveShell.ast_node_interactivity = "all"
display(HTML(data="""<style> div#notebook-container{width:95%;}</style>"""))

import spacy
nlp = spacy.load("en_core_web_sm")


Đầu tiên là quá trình tiền xử lý dữ liệu thô bao gồm các bước xử lý cơ bản để làm sạch dữ liệu. Chúng ta sẽ định dạng lại dữ liệu một cách hợp lý và loại bỏ các giá trị nan có trong tập dữ liệu thô. 

In [22]:
print(data.shape)
data.describe()

data.drop_duplicates(subset=["content"],inplace=True)
data.dropna(inplace=True)
data.reset_index(drop=True,inplace=True)


(580, 4)


,link,title,time,content
count,580,580,580,580
unique,579,579,314,574
top,https://dantri.com.vn/kinh-doanh/nha-dau-tu-ch...,Nhà đầu tư chi 1.200 tỷ đồng mua cổ phiếu công...,25/03/2024,[]
freq,2,2,6,6


In [23]:
pip install underthesea

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     --------------------------- ---------- 30.7/42.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/20.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/20.9 MB 1.7 MB/s eta 0:00:12
   ---------------------------------------- 0.2/20.9 MB 3.1 MB/s eta 0:00:07
    --------------------------------------- 0.5/20.9 MB 3.7 MB/s eta 0:00:06
   - -------------------------------------- 1.0/20.9 MB 5.7 MB/s eta 0:00:04
   --- ------------------------------------ 1.7/20.9 MB 7.6 MB/s eta 0:00:03
   ---- ----------------------------------- 2.4/20.9 MB 9.0 MB/s eta 0:00:03
   ------ --------------------------------- 3.4/20.9 MB 10.7 MB/s eta 0:00:02
   --------- ------------------------------ 4.8/20.9 MB 12.8 MB/s eta 0:00:02
   ----------- ---------------------------- 6.1/20.9 MB 14.4 MB/s eta 0:00:02
  

Trong Tiếng Anh có rất nhiều từ nối và được sử dụng rất thường xuyên như "and", "the" và "a". Khi thực hiện việc thống kê trên văn bản, những từ này sẽ mang lại rất nhiều nhiễu vì chúng xuất hiện thường xuyên hơn các từ khác. Một số pineline về NLP sẽ gắn cờ chúng là các từ dừng (stop words) - nghĩa là các từ mà bạn có thể sẽ muốn lọc ra trước khi thực hiện bất kỳ các phân tích thống kê nào. Tương tự với Tiếng Việt cũng có rất nhiều stop words, chúng ta cần phải loại bỏ chúng:

In [66]:
import pandas as pd
data = pd.read_csv("data_dantri_chungkhoan.csv")

In [67]:
# Đọc tệp vietnamesestopword.txt và lưu các từ dừng vào một danh sách
stopwords = []
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.read().splitlines()

print(f'Number of stopwords: {len(stopwords)}')
print(f'First 10 stopwords: {stopwords[:10]}')


Number of stopwords: 1942
First 10 stopwords: ['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy']


Tiếp theo, chúng ta phân tách dữ liệu văn bản của mình thành đơn vị nhỏ nhất được gọi là "tokens" hoặc "words". Việc phân tích các đoạn văn dài khá khó khăn nên trước tiên chúng ta tách các đoạn văn thành các dòng riêng biệt sau đó các dòng tách thành các từ và xóa stop words:

In [68]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
import re
def clean_para(content):
    """Làm sạch văn bản bằng cách loại bỏ dấu câu, chuyển đổi thành chữ thường và loại bỏ stopwords."""
    # Kiểm tra nếu van_ban không phải là chuỗi, thì chuyển thành chuỗi
    if not isinstance(content, str):
        content = ' '.join(content)
    # Loại bỏ dấu câu
    content = re.sub(r'[^\w\s]', '', content)
    # Chuyển đổi thành chữ thường
    content = content.lower()
    # Loại bỏ stopwords
    tu_vung = content.split()
    tu_vung_sach = [tu for tu in tu_vung if tu not in stopwords]
    return ' '.join(tu_vung_sach)

In [70]:
data['content_cleaned'] = data['content'].apply(clean_para)
print(data[['content_cleaned']])

                                       content_cleaned
0    kết phiên giao dịch 225 mỹ cổ phiếu nvidia 05 ...
1    rung lắc phiên tiếp tục chỉnh đầu phiên chiều ...
2    đỗ quang vinh phó chủ tịch hội đồng quản trị n...
3    phiên 215 vnindex chỉnh nhẹ 044 tương ứng 003 ...
4    mặc rung lắc giằng co suốt phiên giao dịch hôm...
5    phiên giao dịch thị trường tiếp tục đi sắc xan...
6    giúp đầu tư cá nhân kinh nghiệm khả năng thực ...
7    thị trường diễn biến tích cực phiên 165 khoản ...
8    vnindex tiếp tục nới rộng đà phiên chiều 155 b...
9    phiên giao dịch chiều 145 giằng co trạng thái ...
10   khoản tương đương phiên hôm thị trường chứng k...
11   đầu phiên 135 cổ phiếu qcg công ty cổ quốc cườ...
12   thị trường chỉnh phiên 135 vnindex đánh rơi 43...
13   tra ủy ban chứng khoán ubcknn ban hành quyết đ...
14   thị trường quanh ngưỡng 1250 rung lắc phiên lù...
15   công ty cổ đầu tư dịch vụ tài hoàng huy công b...
16   giới tài câu ngạn ngữ nổi tiếng sell in may an...
17   diễn 

In [72]:
def split_para(content):
    """Tách một đoạn văn thành danh sách các câu."""
    if not isinstance(content, str):
        content = ' '.join(content)
    sentence = nltk.sent_tokenize(content)
    return sentence
data['sentences'] = data['content_cleaned'].apply(split_para)
print(data[['sentences']])

                                             sentences
0    [kết phiên giao dịch 225 mỹ cổ phiếu nvidia 05...
1    [rung lắc phiên tiếp tục chỉnh đầu phiên chiều...
2    [đỗ quang vinh phó chủ tịch hội đồng quản trị ...
3    [phiên 215 vnindex chỉnh nhẹ 044 tương ứng 003...
4    [mặc rung lắc giằng co suốt phiên giao dịch hô...
5    [phiên giao dịch thị trường tiếp tục đi sắc xa...
6    [giúp đầu tư cá nhân kinh nghiệm khả năng thực...
7    [thị trường diễn biến tích cực phiên 165 khoản...
8    [vnindex tiếp tục nới rộng đà phiên chiều 155 ...
9    [phiên giao dịch chiều 145 giằng co trạng thái...
10   [khoản tương đương phiên hôm thị trường chứng ...
11   [đầu phiên 135 cổ phiếu qcg công ty cổ quốc cư...
12   [thị trường chỉnh phiên 135 vnindex đánh rơi 4...
13   [tra ủy ban chứng khoán ubcknn ban hành quyết ...
14   [thị trường quanh ngưỡng 1250 rung lắc phiên l...
15   [công ty cổ đầu tư dịch vụ tài hoàng huy công ...
16   [giới tài câu ngạn ngữ nổi tiếng sell in may a...
17   [diễn

In [74]:
def split_sentence(sentence):
    """Tách một câu thành danh sách các từ."""
    word = nltk.word_tokenize(sentence)
    return word
data['words'] = data['sentences'].apply(lambda x: [split_sentence(sentence) for sentence in split_para(x)])
print(data[['words']])

                                                 words
0    [[kết, phiên, giao, dịch, 225, mỹ, cổ, phiếu, ...
1    [[rung, lắc, phiên, tiếp, tục, chỉnh, đầu, phi...
2    [[đỗ, quang, vinh, phó, chủ, tịch, hội, đồng, ...
3    [[phiên, 215, vnindex, chỉnh, nhẹ, 044, tương,...
4    [[mặc, rung, lắc, giằng, co, suốt, phiên, giao...
5    [[phiên, giao, dịch, thị, trường, tiếp, tục, đ...
6    [[giúp, đầu, tư, cá, nhân, kinh, nghiệm, khả, ...
7    [[thị, trường, diễn, biến, tích, cực, phiên, 1...
8    [[vnindex, tiếp, tục, nới, rộng, đà, phiên, ch...
9    [[phiên, giao, dịch, chiều, 145, giằng, co, tr...
10   [[khoản, tương, đương, phiên, hôm, thị, trường...
11   [[đầu, phiên, 135, cổ, phiếu, qcg, công, ty, c...
12   [[thị, trường, chỉnh, phiên, 135, vnindex, đán...
13   [[tra, ủy, ban, chứng, khoán, ubcknn, ban, hàn...
14   [[thị, trường, quanh, ngưỡng, 1250, rung, lắc,...
15   [[công, ty, cổ, đầu, tư, dịch, vụ, tài, hoàng,...
16   [[giới, tài, câu, ngạn, ngữ, nổi, tiếng, sell,...
17   [[diễ

In [75]:
data.head()

,link,title,time,content,content_cleaned,sentences,words
0,https://dantri.com.vn/kinh-doanh/nvidia-cong-b...,"Nvidia công bố doanh thu đột biến nhờ AI, cổ p...",23/05/2024,"Kết phiên giao dịch ngày 22/5 (theo giờ Mỹ), c...",kết phiên giao dịch 225 mỹ cổ phiếu nvidia 05 ...,[kết phiên giao dịch 225 mỹ cổ phiếu nvidia 05...,"[[kết, phiên, giao, dịch, 225, mỹ, cổ, phiếu, ..."
1,https://dantri.com.vn/kinh-doanh/xang-len-gia-...,"Xăng lên giá, cổ phiếu xăng dầu bùng nổ",23/05/2024,Rung lắc mạnh trong phiên sáng và tiếp tục điề...,rung lắc phiên tiếp tục chỉnh đầu phiên chiều ...,[rung lắc phiên tiếp tục chỉnh đầu phiên chiều...,"[[rung, lắc, phiên, tiếp, tục, chỉnh, đầu, phi..."
2,https://dantri.com.vn/kinh-doanh/dai-thieu-gia...,Đại thiếu gia nhà bầu Hiển tăng sở hữu tại SHB,23/05/2024,Ông Đỗ Quang Vinh - Phó chủ tịch Hội đồng quản...,đỗ quang vinh phó chủ tịch hội đồng quản trị n...,[đỗ quang vinh phó chủ tịch hội đồng quản trị ...,"[[đỗ, quang, vinh, phó, chủ, tịch, hội, đồng, ..."
3,https://dantri.com.vn/kinh-doanh/co-phieu-tra-...,"""Cổ phiếu trà đá"" tăng ồ ạt, một mã ngân hàng ...",21/05/2024,"Trong phiên 21/5, chỉ số chính VN-Index điều c...",phiên 215 vnindex chỉnh nhẹ 044 tương ứng 003 ...,[phiên 215 vnindex chỉnh nhẹ 044 tương ứng 003...,"[[phiên, 215, vnindex, chỉnh, nhẹ, 044, tương,..."
4,https://dantri.com.vn/kinh-doanh/co-phieu-nha-...,"Cổ phiếu nhà Cường ""Đô La"" trở lại; tiền ồ ạt ...",20/05/2024,Mặc dù rung lắc và giằng co nhưng trong suốt p...,mặc rung lắc giằng co suốt phiên giao dịch hôm...,[mặc rung lắc giằng co suốt phiên giao dịch hô...,"[[mặc, rung, lắc, giằng, co, suốt, phiên, giao..."
